In [ ]:
## Standard Stuff
import numpy as np
import xarray as xr
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import dask.array as da
import numcodecs

## HEALPix Specific
import healpix as hp
import easygems.healpix as egh
import easygems.remap as egr

import intake     # For catalogs
import zarr       

In [ ]:
catfn='/home/tmerlis/hackathon/xsh24_scream_main.yaml'

combo_cat = intake.open_catalog(catfn)

# 'coarse' is an online coarse-graining of 8 neighboring grid cells ~25km
# xsh24 = X-SHiELD 2024 model version, unpublished
# xsh21 = X-SHiELD 2021 model version, many articles including Cheng et al. 2022 GRL
print (list(combo_cat)) 

In [ ]:
# select zoom level and the part of the combined catalog you're interested in
# coarse stores are available at zoom 7 ~50km and lower
zoom_select = 7
ds = combo_cat.xsh24_coarse(zoom=zoom_select).to_dask()
# attach coordinates; otherwise can't use lat and lon and selecting regions or taking a zonal mean won't work
ds = ds.pipe(egh.attach_coords)

# native stores are available at zoom 10 ~6.4km and lower, fewer variables and all are 2d
dsn = combo_cat.xsh24_native(zoom=10).to_dask()
dsn = dsn.pipe(egh.attach_coords)

# we downloaded output from DOE's SCREAM model to stellar
# conceivably, we could add to our local collection of models during the week
# or augment the catalog with observational datasets
ds_scream = combo_cat.scream_ne120(zoom=zoom_select).to_dask()
ds_scream = ds_scream.pipe(egh.attach_coords)

In [ ]:
# take a look at what variables are healpix-ified
# there are many more diagnostic variables that weren't part of the data request
ds

In [ ]:
def worldmap(var, **kwargs):
    #projection = ccrs.Robinson(central_longitude=-135.5808361)
    projection = ccrs.Robinson(central_longitude=0)
    fig, ax = plt.subplots(
        figsize=(8, 4), subplot_kw={"projection": projection}, constrained_layout=True
    )
    ax.set_global()

    hpshow = egh.healpix_show(var, ax=ax, **kwargs)
    cbar = plt.colorbar(hpshow, ax=ax, orientation='vertical', 
                    pad=0.05, shrink=0.8)


In [ ]:
%%time

worldmap(ds.uas.sel(time=slice('2020-01-01', '2020-12-31')).mean(dim='time'))

In [ ]:
# SCREAM
worldmap(ds_scream.uas.sel(time=slice('2020-01-01', '2020-12-31')).mean(dim='time')) 

In [ ]:
%%time

# native output at a higher zoom; this is just a one-month mean 
worldmap(dsn.uas.sel(time=slice('2020-01-01', '2020-01-31')).mean(dim='time'))

In [ ]:
%%time

# area weighted global mean is easy in healpix, could use a lower zoom to go faster
plt.plot(ds.pr.mean("cell")*86400)

In [ ]:
%%time

# snapshot
var='ua'
tm='2020-01-05'
tmp=ds[var].sel(time=tm)[0]

tmp

# zonal mean 
zm = (
    tmp
    .groupby("lat")
    .mean()
).compute()

zm.plot()
plt.gca().invert_yaxis()

In [ ]:
# trying out a different colormap
worldmap(ds.pr.sel(time=slice('2020-01-01', '2020-01-31')).mean(dim='time')*86400, cmap="Blues")

In [ ]:
worldmap(ds_scream.pr.sel(time=slice('2020-01-01', '2020-01-31')).mean(dim='time')*86400, cmap="Blues")